In [63]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import os
from mne.io import read_raw_edf
from mne.time_frequency import psd_array_multitaper
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder


In [28]:
path = '/Users/splaw/Desktop/dyplom/eegParkinsonCognitiveDisorders/EEG'
# stwórz listę plików EDF w folderze
print(edf_files)

['PD129.edf', 'PD130.edf', 'PD131.edf', 'PD132.edf', 'PD133.edf', 'PD134.edf', 'PD135.edf', 'PD136.edf', 'PD137.edf', 'PD138.edf', 'PD139.edf', 'PD140.edf', 'PD141.edf', 'PD142.edf', 'PD143.edf', 'PD144.edf', 'PD145.edf', 'PD146.edf', 'PD147.edf', 'PD148.edf', 'PD149.edf', 'PD150.edf', 'PD151.edf', 'PD152.edf', 'PD153.edf', 'PD154.edf', 'PD155.edf', 'PD156.edf', 'PD157.edf', 'PD158.edf', 'PD159.edf', 'PD160.edf', 'PD161.edf', 'PD162.edf', 'PD163.edf', 'PD164.edf', 'PD165.edf', 'PD166.edf', 'PD167.edf', 'PD170.edf', 'PD171.edf', 'PD172.edf', 'PD173.edf', 'PD174.edf', 'PD175.edf', 'PD176.edf', 'PD177.edf', 'PD178.edf', 'PD180.edf', 'PD181.edf', 'PD182.edf', 'PD183.edf', 'PD184.edf', 'PD185.edf', 'PD186.edf', 'PD187.edf', 'PD188.edf', 'PD189.edf', 'PD190.edf', 'PD191.edf', 'PD192.edf', 'PD193.edf', 'PD194.edf', 'PD195.edf', 'PD196.edf', 'PD197.edf', 'PD198.edf', 'PD199.edf', 'PD200.edf', 'PD201.edf', 'PD202.edf', 'PD203.edf', 'PD204.edf', 'PD205.edf', 'PD206.edf', 'PD207.edf', 'PD208.edf'

In [44]:
data = []
for filename in os.listdir(path):
    if filename.endswith('.edf'):
        filepath = os.path.join(path, filename)
        raw = mne.io.read_raw_edf('PD129.edf',preload=True)
        events = mne.make_fixed_length_events(raw, duration=1.0)
        raw.filter(1, 40, fir_design='firwin')
        epochs = mne.Epochs(raw, events, tmin=0, tmax=2, baseline=None)
        epo_spectrum = epochs.compute_psd(method='multitaper', fmin=5, fmax=30,
                picks='eeg')
        psds, freqs = epo_spectrum.get_data(return_freqs=True)
        data.append(psds)

Extracting EDF parameters from c:\Users\splaw\Desktop\dyplom\eegParkinsonCognitiveDisorders\PD129.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 755199  =      0.000 ...  1474.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1691 samples (3.303 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.6s remaining:    0.0s


Not setting metadata
1475 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1475 events and 1025 original time points ...


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.8s finished


2 bad epochs dropped
    Using multitaper spectrum estimation with 7 DPSS windows
Extracting EDF parameters from c:\Users\splaw\Desktop\dyplom\eegParkinsonCognitiveDisorders\PD129.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 755199  =      0.000 ...  1474.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1691 samples (3.303 sec)



...

2 bad epochs dropped
    Using multitaper spectrum estimation with 7 DPSS windows
Extracting EDF parameters from c:\Users\splaw\Desktop\dyplom\eegParkinsonCognitiveDisorders\PD129.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 755199  =      0.000 ...  1474.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1691 samples (3.303 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
1475 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1475 events and 1025 original time points ...


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.4s finished


2 bad epochs dropped
    Using multitaper spectrum estimation with 7 DPSS windows


In [103]:
# Read the CSV file
data = pd.read_csv("/Users/splaw/Desktop/dyplom/eegParkinsonCognitiveDisorders/dataset.csv", sep=';')

# Select only the 'PD' column as the label
label = data['PD']
df = data.iloc[:, :-1]
label = data['PD']
le = LabelEncoder()
df['PD'] = le.fit_transform(label)



In [104]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df['PD'], test_size=0.2, random_state=42)

# Build the Random Forest Classifier model
rfc = RandomForestClassifier(n_estimators=100, random_state=42)

# Cross-validation of the model
scores = cross_val_score(rfc, X_train, y_train, cv=5)

# Train the model on the entire training set
rfc.fit(X_train, y_train)

# Predict the labels on the testing set
y_pred = rfc.predict(X_test)

# Classification report
print(classification_report(y_test, y_pred))

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\splaw\Desktop\dyplom\eegParkinsonCognitiveDisorders\nazwa_srodowiska\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\splaw\Desktop\dyplom\eegParkinsonCognitiveDisorders\nazwa_srodowiska\Lib\site-packages\sklearn\ensemble\_forest.py", line 345, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\splaw\Desktop\dyplom\eegParkinsonCognitiveDisorders\nazwa_srodowiska\Lib\site-packages\sklearn\base.py", line 584, in _validate_data
    X, y = check_X_y(X, y, **check_params)
           ^^^^^^^